In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score, StratifiedKFold
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso, RidgeCV, LassoCV,LogisticRegressionCV, ElasticNet
from sklearn.kernel_ridge import KernelRidge
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from sklearn.decomposition import PCA
import xgboost as xgb
from xgboost.sklearn import XGBClassifier, XGBRegressor
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers
from keras import optimizers

Using TensorFlow backend.
/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
#Helper functions that I will usually use for a ML project
def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=',')
def write_file(filename, ID, y_test):
    '''
    Function to write a file of predicted results given a filename, ID-numbers, and predicted y-values for test data
    '''
    import csv
    with open(filename,'w') as f:
        f.write('id,actual_wait div 60000\n')
        writer=csv.writer(f,delimiter=',')
        writer.writerows(zip(ID,list((y_test))))
    f.close()
    
def Dimensionality_reduction_PCA(x_train, dimensions):
    '''
    This function performs PCA on training data and returns the
    reduced dimensionality array.
    Inputs: 
        x_train: the training data input
        dimensions: the number of wanted dimensions
    Output: 
        x_train_reduced: the array with reduced dimensionsionality to 
        the number given by the dimensions paramter
    '''
    pca = PCA(n_components = dimensions)
    x_train_reduced = pca.fit_transform(x_train) #fit and transform the training input data
    return x_train_reduced

def data_reduction(x_train, percentage_threshold):
    '''
    This function performs PCA on training data and returns the
    reduced dimensionality array.
    Inputs: 
        x_train: the training data input
        dimensions: the number of wanted dimensions
    Output: 
        x_train_reduced: the array with reduced dimensionsionality to 
        the number given by the dimensions paramter
    '''
    shape = x_train.shape
    delete_cols = [] # list to hold columns to delete

    for i in range(shape[1]):
        col = x_train[:,i]
        unique, counts = np.unique(col, return_counts=True)
        maxPercent = np.max(counts) / shape[0]
        if(maxPercent > percentage_threshold): # if the percentage of a certain class is high enough, then slice.
            delete_cols.append(i)
    return delete_cols

def delete_cols(dataset, delete_cols):
    '''
    Function to delete columns returned by the data_reduction function
    '''
    return np.delete(dataset, delete_cols, 1)

def normalize_data(x_data):
    '''
    Function to normalize data. 
    '''
    new_x = x_data.copy()
    shape = new_x.shape
    for i in range(shape[1]):
        col = new_x[:,i]
        maxVal = np.max(col)
        new_x[:,i] /= maxVal
    return new_x

In [4]:
#Load training data
X_train = load_data('Data/X_train.csv')
X_test = load_data('Data/X_test.csv')
y_train = load_data('Data/y_train.csv')

#The optimized PCA dimensions were found qualitatively through a loop of simulations and comparing cross validation scores
x_train_PCA_reduced = Dimensionality_reduction_PCA(X_train, 3) #Reduce data dimensions
x_test_PCA_reduced = Dimensionality_reduction_PCA(X_test, 3) #Reduce data dimensions

In [5]:
#Scale data to use in certain regression models below
scaler = StandardScaler()
scaler.fit(X_train)

X_train_ST = scaler.transform(X_train)
X_test_ST = scaler.transform(X_test)

In [6]:
#Let's try to do a simple approach with single regressions
#Starting with lasso regression
lasso1 = LassoCV(alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 3, 5, 10,100], cv=5)
#All training data scores were evaluated using 5-fold cross-validation
#Parameters were optimized by running cross validation for multiple parameter combinations in each cell
CV_score_lasso = np.mean(cross_val_score(lasso1,X_train,y_train,cv=5))
print(np.mean(CV_score_lasso))
lasso1.fit(X_train, y_train)

/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model

0.4326155209872621


/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 2, 3, 5, 10, 100], copy_X=True,
    cv=5, eps=0.001, fit_intercept=True, max_iter=1000, n_alphas=100,
    n_jobs=None, normalize=False, positive=False, precompute='auto',
    random_state=None, selection='cyclic', tol=0.0001, verbose=False)

In [8]:
#Next ridge regression
ridge1 = RidgeCV(alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 3, 5, 10,100], cv=5)
#All training data scores were evaluated using 5-fold cross-validation
#Parameters were optimized by running cross validation for multiple parameter combinations in each cell
CV_score_ridge = np.mean(cross_val_score(ridge1,X_train_ST,y_train,cv=5))
print(CV_score_ridge)
ridge1.fit(X_train_ST,y_train)

0.4312441040980386


RidgeCV(alphas=array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 2.e+00, 3.e+00, 5.e+00,
       1.e+01, 1.e+02]),
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [9]:
#Hmm not great, let's try a regularized expression with elastic net
EN = ElasticNet()
EN.set_params(alpha=1.6,l1_ratio=0.5,max_iter=10000)
#All training data scores were evaluated using 5-fold cross-validation
#Parameters were optimized by running cross validation for multiple parameter combinations in each cell
CV_score_EN = np.mean(cross_val_score(EN,X_train_ST,y_train,cv=5))
print(CV_score_EN)
EN.fit(X_train_ST,y_train)

0.3596119916794528


ElasticNet(alpha=1.6, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=10000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [10]:
#Let's try gradient boosting with the LightGBM module
params = {'objective': 'regression', 'num_leaves': 50,
          'learning_rate': 0.05, 'n_estimators': 100,
          'max_bin': 100, 'bagging_fraction': 0.9,
          'bagging_freq': 1, 'feature_fraction': 0.5,
          'feature_fraction_seed': 1, 'bagging_seed': 1,
          'min_data_in_leaf': 10, 'min_sum_hessian_in_leaf': 11}
lgbm1 = lgb.LGBMRegressor(**params)
#All training data scores were evaluated using 5-fold cross-validation
#Parameters were optimized by running cross validation for multiple parameter combinations in each cell
CV_score_lgbm = np.mean(cross_val_score(lgbm1,X_train,y_train,cv=5))
print(CV_score_lgbm)
lgbm1.fit(X_train, y_train)

0.6140667056988848


LGBMRegressor(bagging_fraction=0.9, bagging_freq=1, bagging_seed=1,
       boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       feature_fraction=0.5, feature_fraction_seed=1,
       importance_type='split', learning_rate=0.05, max_bin=100,
       max_depth=-1, min_child_samples=20, min_child_weight=0.001,
       min_data_in_leaf=10, min_split_gain=0.0, min_sum_hessian_in_leaf=11,
       n_estimators=100, n_jobs=-1, num_leaves=50, objective='regression',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [11]:
#Okay, gradient boosting seems to work well
#Let's try another method
GBoost = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=1, min_samples_split=10, 
                                   loss='huber', random_state =5)
#All training data scores were evaluated using 5-fold cross-validation
#Parameters were optimized by running cross validation for multiple parameter combinations in each cell
CV_score_GBoost = np.mean(cross_val_score(GBoost,X_train,y_train,cv=5))
print(CV_score_GBoost)
GBoost.fit(X_train,y_train)

0.5925532268918335


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=4,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             n_iter_no_change=None, presort='auto', random_state=5,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [12]:
#Okay, let's try another gradient boosting method
xgb1 = XGBRegressor(learning_rate=0.5,
                subsample=0.8,
                colsample_bytree=0.8,
                objective = 'reg:linear',
                seed=1,
                gamma=1,
                max_depth=6,
                min_child_weight=5,
                n_estimators=10)
#All training data scores were evaluated using 5-fold cross-validation
#Parameters were optimized by running cross validation for multiple parameter combinations in each cell
CV_score_XGB = np.mean(cross_val_score(xgb1,X_train,y_train,cv=5))
print(np.mean(CV_score_XGB))
xgb1.fit(X_train, y_train)

0.583454248726604


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=1, learning_rate=0.5, max_delta_step=0,
       max_depth=6, min_child_weight=5, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1, silent=True,
       subsample=0.8)

In [13]:
#Ok, let's see what Adaboost can do in comparison to gradient boosting
ABoost = AdaBoostRegressor(n_estimators=10,learning_rate=0.05,loss='square', random_state =5)
#All training data scores were evaluated using 5-fold cross-validation
#Parameters were optimized by running cross validation for multiple parameter combinations in each cell
CV_score_ABoost = np.mean(cross_val_score(ABoost,X_train,y_train,cv=5))
print(CV_score_ABoost)
ABoost.fit(X_train,y_train)

0.3975861680060328


AdaBoostRegressor(base_estimator=None, learning_rate=0.05, loss='square',
         n_estimators=10, random_state=5)

In [ ]:
#I found that the decision tree model below did not work well with data
#DFmodel1 = DecisionTreeRegressor(criterion='mse')
#DFmodel1.set_params(min_samples_leaf=20,max_depth=110,max_features='log2')
#CV_score_DF = np.mean(cross_val_score(DFmodel1,X_train_ST,y_train))
#print(CV_score_DF)
#DFmodel1.fit(X_train_ST,y_train)

In [14]:
#Since the decision tree model did not work well - let's train a couple random forests
RFmodel1 = RandomForestRegressor(criterion='mse')
RFmodel1.set_params(n_estimators=1000,min_samples_leaf=20,max_depth=110,max_features='log2')
#All training data scores were evaluated using 5-fold cross-validation
#Parameters were optimized by running cross validation for multiple parameter combinations in each cell
CV_score_RF1 = np.mean(cross_val_score(RFmodel1,X_train,y_train,cv=5))
print(CV_score_RF1)
RFmodel1.fit(X_train,y_train)

0.57699988399546


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=20, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [31]:
#I found that the first random forest model did better than the second
#RFmodel2 = RandomForestRegressor(criterion='mse')
#RFmodel2.set_params(n_estimators=1500,min_samples_leaf=20,max_depth=110,max_features='log2')
#CV_score_RF2 = np.mean(cross_val_score(RFmodel2,X_train,y_train,cv=5))
#print(CV_score_RF2)
#RFmodel2.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=20, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1500, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [15]:
#Okay none of the models really did that well and I was running out of time
#I tried to remove repetitive data columns using the helper functions above but it did not help the trainnig scores 

#I decided to stack my models
#Here I'm stacking my test data prediction from all of my models
lasso_pred = lasso1.predict(X_test)
ridge_pred = ridge1.predict(X_test_ST)
EN_pred = EN.predict(X_test_ST)
LGBM_pred = lgbm1.predict(X_test)
GBoost_pred = GBoost.predict(X_test)
XGB_pred = xgb1.predict(X_test)
ABoost_pred = ABoost.predict(X_test)
RF1_pred = RFmodel1.predict(X_test)
X_pred_stack = np.stack((lasso_pred,ridge_pred,ABoost_pred,RF1_pred,GBoost_pred,XGB_pred,EN_pred), axis=-1)

In [16]:
#Here I'm stacking my training data predictions from all of my models
RF1_pred = np.empty_like(y_train)
GBoost_pred = np.empty_like(y_train)
ABoost_train = np.empty_like(y_train)
Lasso_pred = np.empty_like(y_train)
Ridge_pred = np.empty_like(y_train)
LGBM_train = np.empty_like(y_train)
DF_train = np.empty_like(y_train)
XGB_train = np.empty_like(y_train)
EN_train = np.empty_like(y_train)
cv = StratifiedKFold(n_splits=5, shuffle=True)
i = 0
for train, val in cv.split(X_train, y_train):
    i += 1
    print('Fold %d' % i)
    RF1_pred[val] = RFmodel1.predict(X_train[val])
    GBoost_pred[val] = GBoost.predict(X_train[val])
    ABoost_train[val] = ABoost.predict(X_train_ST[val])
    Lasso_pred[val] = lasso1.predict(X_train[val])
    Ridge_pred[val] = ridge1.predict(X_train_ST[val])
    LGBM_train[val] = lgbm1.predict(X_train[val])
    XGB_train[val] = xgb1.predict(X_train[val])
    EN_train[val] = EN.predict(X_train_ST[val])
    
X_train_stack = np.stack((Lasso_pred,Ridge_pred,ABoost_train,RF1_pred,GBoost_pred,XGB_train,EN_train), axis=-1)

/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


In [17]:
#I transform my stacked data
scaler = StandardScaler()
scaler.fit(X_train_stack)
X_train_stack_ST = scaler.transform(X_train_stack)
X_test_stack_ST = scaler.transform(X_pred_stack)

In [20]:
#I now try different metamodel approaches with my stacked models
#First is a ridge model
ridge_meta = RidgeCV(alphas=[1e-4, 1e-3, 1e-2, 1e-1, 5e-1, 1, 1.5, 2, 3, 5, 10, 30, 50, 100], cv=5)
CV_score_final = np.mean(cross_val_score(ridge_meta,X_train_stack_ST,y_train,cv=5))
print(np.mean(CV_score_final))
ridge_meta.fit(X_train_stack_ST, y_train)

0.7339713947782142


RidgeCV(alphas=array([1.0e-04, 1.0e-03, 1.0e-02, 1.0e-01, 5.0e-01, 1.0e+00, 1.5e+00,
       2.0e+00, 3.0e+00, 5.0e+00, 1.0e+01, 3.0e+01, 5.0e+01, 1.0e+02]),
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [19]:
#Ok, I realize that my model did much better when stacked
#I now try another metamodel approach with an elastic net
#It did slightly better
EN_meta = ElasticNet()
EN_meta.set_params(alpha=0.05,l1_ratio=0.9,max_iter=100)
CV_score_EN_meta = np.mean(cross_val_score(EN_meta,X_train_stack_ST,y_train,cv=5))
print(np.mean(CV_score_EN_meta))
EN_meta.fit(X_train_stack_ST,y_train)

/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model

0.7338677404377764


/Users/evascheller/anaconda3/envs/CS155/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


ElasticNet(alpha=0.05, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=100, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [21]:
#Last I try to parse the stacked models into a NN
#I try for a while to optimize parameters and number of layers, but the NN is not doing too well on the accuracy score
#I had previously tried using NN to model data, and likewise it did not perform well on data, so I abandon this approach
y_train_onehotencoded = to_categorical(y_train)

model = Sequential()
model.add(Dense(500, input_dim=X_train_stack.shape[1]))
model.add(Activation('relu'))
          
model.add(Dense(500))
model.add(Activation('relu'))

model.add(Dense(500))
model.add(Activation('relu'))

model.add(Dense(525))
model.add(Activation("sigmoid"))

model.summary()

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_stack, y_train_onehotencoded, batch_size=64, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               4000      
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_2 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_3 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 525)               263025    
__________

In [22]:
#Out of all my models the ridge meta model of my stacked models worked the best, so I chose this for my prediction
y_test = ridge_meta.predict(X_test_stack_ST)

#I write out my predictions to a file
ID = []
for i in range(len(y_test)):
    ID.append(i)
write_file('Data/y_test.csv',ID,y_test)